<a href="https://colab.research.google.com/github/kapuskaFaizan/ML-python-notebooks/blob/master/rec_surprise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Importing the data

movie_titles_genre = pd.read_csv('movies.csv')
movie_titles_genre.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
data = pd.read_csv('ratings.csv')
data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
d2 = pd.read_csv('links.csv')
d2.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [ ]:
d3 = pd.read_csv('tags.csv')
d3.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [ ]:
#data pre processing

data = data.merge(movie_titles_genre,on='movieId', how='left')

In [ ]:
data.head(10)

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
5,1,70,3.0,964982400,From Dusk Till Dawn (1996),Action|Comedy|Horror|Thriller
6,1,101,5.0,964980868,Bottle Rocket (1996),Adventure|Comedy|Crime|Romance
7,1,110,4.0,964982176,Braveheart (1995),Action|Drama|War
8,1,151,5.0,964984041,Rob Roy (1995),Action|Drama|Romance|War
9,1,157,5.0,964984100,Canadian Bacon (1995),Comedy|War


In [ ]:
Average_ratings = pd.DataFrame(data.groupby('title')['rating'].mean())
Average_ratings.head(10)

,rating
title,
'71 (2014),4.000000
'Hellboy': The Seeds of Creation (2004),4.000000
'Round Midnight (1986),3.500000
'Salem's Lot (2004),5.000000
'Til There Was You (1997),4.000000
'Tis the Season for Love (2015),1.500000
"'burbs, The (1989)",3.176471
'night Mother (1986),3.000000
(500) Days of Summer (2009),3.666667


In [ ]:
Average_ratings['Total Ratings'] = pd.DataFrame(data.groupby('title')['rating'].count())
Average_ratings.head(10)

,rating,Total Ratings
title,,
'71 (2014),4.000000,1
'Hellboy': The Seeds of Creation (2004),4.000000,1
'Round Midnight (1986),3.500000,2
'Salem's Lot (2004),5.000000,1
'Til There Was You (1997),4.000000,2
'Tis the Season for Love (2015),1.500000,1
"'burbs, The (1989)",3.176471,17
'night Mother (1986),3.000000,1
(500) Days of Summer (2009),3.666667,42


In [ ]:
movie_user = data.pivot_table(index='userId',columns='title',values='rating')

In [ ]:
correlations = movie_user.corrwith(movie_user['Toy Story (1995)'])
correlations.head()

/home/kapuska/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/home/kapuska/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


title
'71 (2014)                                NaN
'Hellboy': The Seeds of Creation (2004)   NaN
'Round Midnight (1986)                    NaN
'Salem's Lot (2004)                       NaN
'Til There Was You (1997)                 NaN
dtype: float64

In [ ]:
recommendation = pd.DataFrame(correlations,columns=['Correlation'])
recommendation.dropna(inplace=True)
recommendation = recommendation.join(Average_ratings['Total Ratings'])
recommendation.head()

,Correlation,Total Ratings
title,,
"'burbs, The (1989)",0.240563,17
(500) Days of Summer (2009),0.353833,42
*batteries not included (1987),-0.427425,7
10 Cent Pistol (2015),1.000000,2
10 Cloverfield Lane (2016),-0.285732,14


In [ ]:
recc = recommendation[recommendation['Total Ratings']>100].sort_values('Correlation',ascending=False).reset_index()

In [ ]:
recc = recc.merge(movie_titles_genre,on='title', how='left')
recc.head(10)

,title,Correlation,Total Ratings,movieId,genres
0,Toy Story (1995),1.000000,215,1,Adventure|Animation|Children|Comedy|Fantasy
1,"Incredibles, The (2004)",0.643301,125,8961,Action|Adventure|Animation|Children|Comedy
2,Finding Nemo (2003),0.618701,141,6377,Adventure|Animation|Children|Comedy
3,Aladdin (1992),0.611892,183,588,Adventure|Animation|Children|Comedy|Musical
4,"Monsters, Inc. (2001)",0.490231,132,4886,Adventure|Animation|Children|Comedy|Fantasy
5,Mrs. Doubtfire (1993),0.446261,144,500,Comedy|Drama
6,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",0.438237,120,4973,Comedy|Romance
7,American Pie (1999),0.420117,103,2706,Comedy|Romance
8,Die Hard: With a Vengeance (1995),0.410939,144,165,Action|Crime|Thriller
9,E.T. the Extra-Terrestrial (1982),0.409216,122,1097,Children|Drama|Sci-Fi


In [ ]:

data = data[['userId', 'movieId', 'rating', 'timestamp']]
data = data.iloc[:, :-1]

In [ ]:
from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(data[['userId', 'movieId', 'rating']], reader)

In [ ]:

from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25)

In [ ]:

from surprise import SVD, accuracy
algo = SVD()
algo.fit(trainset)

In [ ]:

predictions = algo.test(testset)

In [ ]:
print('for svd')
print('the error & accuracy')
from surprise import accuracy
accuracy.rmse(predictions)

for svd
the error & accuracy
RMSE: 0.8747


0.8747074642941514

In [ ]:
# clustering

from surprise.prediction_algorithms.knns import KNNBasic

In [ ]:
c = KNNBasic()

In [ ]:
c.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
pred = c.test(testset)

In [ ]:
print('As for knn the accuracy is far better so we choose this one')
from surprise import accuracy
accuracy.rmse(pred)

as for knn the accuracy is far better so we choose this one
RMSE: 0.9482


0.9482096221369373

In [ ]:
len(pred)

25209

In [ ]:
type(trainset)

surprise.trainset.Trainset

In [ ]:
result = pd.DataFrame(pred)


In [ ]:
result.drop(columns = {'details'}, inplace = True)

In [ ]:
result.head()

,uid,iid,r_ui,est
0,603,1394,5.0,3.919975
1,434,1517,4.0,3.649209
2,182,1687,4.0,3.144087
3,22,49272,5.0,3.872095
4,448,5334,1.0,2.000000


In [ ]:
IDs: 42, 314 and 444;

In [ ]:
print('#'*60)
print(' Recommendations for user with id 42' + '|| iid is the id of movie')

result.loc[result['uid'] == 42]

############################################################
 recommendations for user with id 42 iid is the id of movie


,uid,iid,r_ui,est
100,42,223,5.0,4.151005
120,42,236,3.0,3.520862
342,42,144,4.0,3.267242
377,42,2028,5.0,4.208931
500,42,1485,4.0,3.039480
...,...,...,...,...
24587,42,1687,3.0,3.215533
24680,42,3441,4.0,3.499239
24708,42,2605,4.0,3.071964
24718,42,2406,3.0,3.453426


In [ ]:
print('#'*60)
print(' Recommendations for user with id 314' + '|| iid is the id of movie')

result.loc[result['uid'] == 314]

############################################################
 Recommendations for user with id 314 iid is the id of movie


,uid,iid,r_ui,est
981,314,172,1.0,2.699237
2036,314,316,2.0,3.352732
2037,314,524,4.0,3.630203
2216,314,780,3.0,3.522339
2489,314,60,3.0,3.171913
2654,314,592,3.0,3.442056
3434,314,1367,3.0,2.981927
4819,314,434,1.0,2.618921
7559,314,474,3.0,3.624025
8002,314,314,4.0,3.568770


In [ ]:
# est is the estimated rating in the dataframe
# r_ui is the actual rating
# iid is the movie id
# uid is the user id


print('#'*60)
print(' Recommendations for user with id 444' + '|| iid is the id of movie')

result.loc[result['uid'] == 444]

############################################################
 Recommendations for user with id 444|| iid is the id of movie


,uid,iid,r_ui,est
2358,444,344,3.0,3.248394
4496,444,527,5.0,4.320655
6394,444,185,3.0,3.073005
6900,444,273,4.0,3.035652
8472,444,356,5.0,4.244824
8533,444,173,2.0,2.694218
8994,444,595,4.0,3.680961
11795,444,350,3.0,3.301335
12817,444,454,3.0,3.361017
16677,444,592,3.0,3.538141


In [ ]:
print('the recommendations seem perfct as the error rate is very less')


the recommendations seem perfct as the error rate is very less
